Important note: for plotting the bubble map you will need to find the longitude and 
latitude for each location and the size of the bubble will indicate the count of the students from the location

In [13]:
import folium     # مكتبة لإنشاء خرائط تفاعلية
import math      
import pandas as pd 

df = pd.read_csv("data_science_student_marks.csv")

# حساب عدد الطلاب في كل موقع (كم طالب في كل location)
data = df["location"].value_counts().reset_index()
data.columns = ["location", "count"]  

# قاموس يحتوي على أسماء المدن مع إحداثياتها (خط العرض والطول)
coords = {
    "Sydney": (-33.8688, 151.2093),
    "Tokyo": (35.6762, 139.6503),
    "Berlin": (52.5200, 13.4050),
    "London": (51.5074, -0.1278),
    "Paris": (48.8566, 2.3522),
    "New York": (40.7128, -74.0060),
    "Toronto": (43.6532, -79.3832),
    "Muscat": (23.5880, 58.3829),
    "Dubai": (25.276987, 55.296249),
    "Mumbai": (19.0760, 72.8777)
}

# ربط كل موقع بإحداثياته من القاموس (lat: خط العرض، lon: خط الطول)
data[["lat", "lon"]] = data["location"].map(coords).apply(pd.Series)

# إنشاء خريطة جديدة بخلفية العالم (OpenStreetMap)
m = folium.Map(location=[20, 0], zoom_start=2)

# رسم دوائر على الخريطة تمثل عدد الطلاب في كل مدينة
for row in data.dropna().itertuples():  # itertuples تسهّل المرور على الصفوف
    folium.Circle(
        location=[row.lat, row.lon],  # إحداثيات المدينة
        radius=row.count * 20000 * math.cos(row.lat * math.pi / 180),  # حجم الدائرة يعتمد على عدد الطلاب
        color="crimson",            
        fill=True,                  
        fill_opacity=0.6,             # شفافية التعبئة
        popup=f"{row.location}: {row.count} students"  # نص يظهر عند النقر على الدائرة
    ).add_to(m)  # إضافة الدائرة إلى الخريطة
m # عرض الخريطة

In [14]:
import folium
import pandas as pd


df = pd.read_csv("data_science_student_marks.csv")

# حساب عدد الطلاب في كل موقع
data = df["location"].value_counts().reset_index()
data.columns = ["location", "student_count"]

# إحداثيات يدوية لأهم المدن
coords = {
    "Sydney": (-33.8688, 151.2093),
    "Tokyo": (35.6762, 139.6503),
    "Berlin": (52.5200, 13.4050),
    "London": (51.5074, -0.1278),
    "Paris": (48.8566, 2.3522),
    "New York": (40.7128, -74.0060),
    "Toronto": (43.6532, -79.3832),
    "Muscat": (23.5880, 58.3829),
    "Dubai": (25.276987, 55.296249),
    "Mumbai": (19.0760, 72.8777)
}

# إضافة الإحداثيات إلى البيانات
data["lat"] = data["location"].map(lambda x: coords.get(x, (None, None))[0])
data["lon"] = data["location"].map(lambda x: coords.get(x, (None, None))[1])

# حذف المواقع التي لا تحتوي على إحداثيات
data = data.dropna(subset=["lat", "lon"])

# إنشاء خريطة فارغة
m = folium.Map(location=[20, 0], tiles="OpenStreetMap", zoom_start=2)

# إضافة علامات (Markers) لكل موقع
for i in range(len(data)):
    folium.Marker(
        location=[data.iloc[i]["lat"], data.iloc[i]["lon"]],
        popup=f"{data.iloc[i]['location']}: {data.iloc[i]['student_count']} students"
    ).add_to(m)

m